In [44]:
import pandas as pd
import numpy as np

In [84]:
# definir arquivo Pickle de entrada
arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df = pd.read_pickle(arquivo)
df.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,1,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,1,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [85]:
df.shape

(9726, 17)

In [86]:
df.drop(df[df['SEXO'] > 1].index, inplace=True)

In [87]:
df.shape

(8711, 17)

### Decomposição com PCA

In [90]:
#from sklearn.decomposition import PCA
#X_reduced = PCA(n_components=4).fit_transform(df.drop(['SEXO'], axis=1))
#X_reduced[:5]

In [91]:
#X = pd.DataFrame(X_reduced, index=df.index)
#y = df['SEXO'] # apenas a coluna alvo
#print('Forma dos dados:', X.shape, y.shape)

### Uso dos dados originais

In [92]:
X = df.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y = df['SEXO'] # apenas a coluna alvo
print('Forma dos dados:', X.shape, y.shape)

Forma dos dados: (8711, 16) (8711,)


In [93]:
X.head()

,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,
MARIA,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANTONIO,15,9,14,15,20,14,1,0,0,0,0,0,0,0,0,0
FRANCISCO,15,3,19,9,3,14,1,18,6,0,0,0,0,0,0,0
JOAO,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
y.head()

PNOME
MARIA        0
JOSE         1
ANTONIO      1
FRANCISCO    1
JOAO         1
Name: SEXO, dtype: uint8

In [95]:
NUMBER_KFOLD_SPLITS = 5 # number of splits in cross-validation
NUMBER_GRID_ITERATIONS = 10 # number of grid iterations to parameters testing
SCORING_METRIC = 'accuracy' # the scoring metric to be used

In [96]:
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

models = {}

def evaluate_model(name, model, X=X, y=y):

  start = datetime.now()
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)
  results = cross_val_score(model, X, y, cv=kfold,
                            scoring=SCORING_METRIC, verbose=1, n_jobs=-1)
  end = datetime.now()

  elapsed = int((end - start).total_seconds() * 1000)
  score = results.mean() * 100
  stddev = results.std() * 100

  models[name] = (model, score, stddev, elapsed)
  print(model, '\nCross-Validation Score: %.2f (+/- %.2f) [%5s ms]' % \
        (score, stddev, elapsed))
  return score, stddev, elapsed

In [97]:
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.model_selection import GridSearchCV

# faz o ajuste fino do modelo, calculando os melhores hiperparâmetros
def fine_tune_model(model, params, X=X, y=y):

  print('\nFine Tuning Model:')
  print(model, "\nparams:", params)
  
  kfold = KFold(n_splits=NUMBER_KFOLD_SPLITS, shuffle=True, random_state=42)

  search = RandomizedSearchCV(model, param_distributions=params, 
                              n_iter=NUMBER_GRID_ITERATIONS, 
                              scoring=SCORING_METRIC, cv=kfold, 
                              verbose=1, n_jobs=-1)
  #search = GridSearchCV(estimator=model, param_grid=params, scoring='accuracy', cv=kfold, verbose=1)

  search.fit(X, y)
  print('\nBest Score: %.2f %%' % (search.best_score_ * 100))
  print('Best Params:', search.best_params_)
  return search

In [98]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(
    random_state=42, multi_class='auto', 
    max_iter=1000, solver='liblinear', C=1.0) # 85.18
evaluate_model('LR', model)

params = dict(
    solver=['liblinear','lbfgs'], 
    C=np.logspace(-3,3,7)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False) 
Cross-Validation Score: 95.26 (+/- 0.21) [12488 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   12.5s finished


In [17]:
# Linear SVM
from sklearn.svm import LinearSVC

model = LinearSVC(random_state=42, max_iter=1000, C=0.001) # 85.27
evaluate_model('LSVM', model)

#from scipy.stats import reciprocal, uniform
#params = dict(C=uniform(1, 10))
params = dict(
    C=np.logspace(-4,3,8)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearSVC(C=0.001, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0) 
Cross-Validation Score: 79.43 (+/- 0.14) [85381 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.4min finished


In [18]:
# K-Nearest Neighbours (KNN)
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier(n_neighbors=11) # 87.45
evaluate_model('KNN', model)

params = dict(
    n_neighbors=[1,3,5,7,9,11]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=11, p=2,
                     weights='uniform') 
Cross-Validation Score: 85.69 (+/- 0.22) [92762 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished


In [99]:
# Linear Discriminant Analysis (LDA)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='svd') # 84.45
evaluate_model('LDA', model)

params = dict(
    solver=['svd','lsqr','eigen']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LinearDiscriminantAnalysis(n_components=None, priors=None, shrinkage=None,
                           solver='svd', store_covariance=False, tol=0.0001) 
Cross-Validation Score: 95.25 (+/- 0.30) [ 1222 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [100]:
# Quadratic Discriminant Analysis (QDA)
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

model = QuadraticDiscriminantAnalysis() # 44.99
evaluate_model('QDA', model)

params = dict()
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


QuadraticDiscriminantAnalysis(priors=None, reg_param=0.0,
                              store_covariance=False, tol=0.0001) 
Cross-Validation Score: 50.24 (+/- 0.65) [  939 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.9s finished


In [101]:
# Naïve Bayes - Gaussian
from sklearn.naive_bayes import GaussianNB

model = GaussianNB(priors=None, var_smoothing=1.0) # 67.62
evaluate_model('GNB', model)

params = dict(
    var_smoothing=np.logspace(0,-9,num=10)
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


GaussianNB(priors=None, var_smoothing=1.0) 
Cross-Validation Score: 95.24 (+/- 0.37) [  778 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


In [102]:
# Naïve Bayes - Bernoulli
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB() # 43.37
evaluate_model('BNB', model)

#params = dict(var_smoothing=np.logspace(0,-9,num=10))
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True) 
Cross-Validation Score: 50.52 (+/- 0.76) [  698 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.7s finished


(50.522470396593754, 0.7597867023173227, 698)

In [103]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(
    random_state=42, criterion='entropy', max_depth=7) # 96.37
evaluate_model('DT', model)

params = dict(
    criterion=['gini','entropy'],
    max_depth=[3,5,7,11,13,17,19]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best') 
Cross-Validation Score: 97.95 (+/- 0.21) [ 1192 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [68]:
# Ensemble - Random Forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(
    random_state=42, max_features='sqrt', n_estimators=500) # 93.86
evaluate_model('RF', model)

params = dict(
    n_estimators=[10,50,100,500], 
    max_features=['auto','sqrt','log2']
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='sqrt', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False) 
Cross-Validation Score: 97.81 (+/- 0.32) [88072 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  1.5min finished


In [104]:
# Ensemble - Ada Boost
from sklearn.ensemble import AdaBoostClassifier

model = AdaBoostClassifier(
    DecisionTreeClassifier(random_state=42, criterion='entropy', max_depth=17), 
    n_estimators=11) # 96.41
evaluate_model('ABDT', model)

params = dict(
    n_estimators=[1,3,5,7,9,11,13]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(class_weight=None,
                                                         criterion='entropy',
                                                         max_depth=17,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort=False,
                                                         random_state=42,
                           

[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    2.7s finished


In [105]:
# Stochastic Gradient Descent (SGD)
from sklearn.linear_model import SGDClassifier

model = SGDClassifier(random_state=42, max_iter=350, tol=0.01) # 77.02
evaluate_model('SGD', model)

params = dict(
    max_iter=[100, 200, 350, 500, 1000], 
    tol=[0.01, 0.1, 1.0]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=350, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=42, shuffle=True, tol=0.01,
              validation_fraction=0.1, verbose=0, warm_start=False) 
Cross-Validation Score: 92.23 (+/- 1.31) [ 5531 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    5.5s finished


In [106]:
# Perceptron
from sklearn.linear_model import Perceptron

model = Perceptron(random_state=42, max_iter=500, tol=0.1) # 75.58
evaluate_model('PCT', model)

params = dict(
    max_iter=[100, 200, 350, 500, 750, 1000], 
    tol=[0.1, 0.01, 0.001]
)
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Perceptron(alpha=0.0001, class_weight=None, early_stopping=False, eta0=1.0,
           fit_intercept=True, max_iter=500, n_iter_no_change=5, n_jobs=None,
           penalty=None, random_state=42, shuffle=True, tol=0.1,
           validation_fraction=0.1, verbose=0, warm_start=False) 
Cross-Validation Score: 85.03 (+/- 9.42) [ 1158 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    1.2s finished


In [107]:
# Extreme Gradient Boosting Machine (XGBM)
from xgboost import XGBClassifier

model = XGBClassifier(
    max_depth=3, min_child_weight=3, gamma=0.4, 
    subsample=0.7, colsample_bytree=0.8, reg_alpha=1e-05) # 97.67
evaluate_model('XGB', model)

params = dict(max_depth=range(3,10,2), min_child_weight=range(1,6,2)) # 97.60
#params = dict(gamma=[i/10.0 for i in range(0,5)]) # 97.68
#params = dict(subsample=[i/10.0 for i in range(6,10)], colsample_bytree=[i/10.0 for i in range(6,10)]) # 97.72
#params = dict(reg_alpha=[1e-5, 1e-2, 0.1, 1, 100]) # 97.74
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.8, gamma=0.4,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=3, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=1e-05, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=0.7, verbosity=1) 
Cross-Validation Score: 97.49 (+/- 0.34) [13030 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   13.0s finished


In [102]:
# LightGBM
from lightgbm import LGBMClassifier
#!pip install lightgbm

model = LGBMClassifier(
    num_leaves=20, max_depth=3, 
    learning_rate=0.1, feature_fraction=1.0) # 97.51
evaluate_model('LGBM', model)

params = dict(num_leaves=range(10,100), max_depth=range(1,30)) # 97.18
#params = dict(learning_rate=[0.01,0.05,0.1,0.5]) # 97.51
#params = dict(feature_fraction=[0.1,0.25,0.5,0.75,1.0]) # 97.62
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


LGBMClassifier(feature_fraction=1.0, max_depth=3, num_leaves=20) 
Cross-Validation Score: 94.40 (+/- 0.49) [ 3188 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.2s finished


In [103]:
# CatBoost
from catboost import CatBoostClassifier
#!pip install catboost

model = CatBoostClassifier(
    silent=True, iterations=250,
    learning_rate=0.05, depth=7) # 97.04
#learning_rate=0.1, iterations=1000, depth=5
evaluate_model('CB', model)

params = dict(
    iterations=[10,50,100,250,500,1000],
    learning_rate=[0.01,0.05,0.1,0.5],
    depth=range(1,11,2)
) 
#fine_tune_model(model, params)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Cross-Validation Score: 94.49 (+/- 0.49) [11935 ms]


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   11.9s finished


In [108]:
names = []
estimators = []
scores = []
stdevs = []
times = []

for key, value in models.items():
    (model, score, stdev, elapsed) = value
    names.append(key)
    estimators.append(model)
    scores.append(score)
    stdevs.append(stdev)
    times.append(elapsed)

In [109]:
results_df = pd.DataFrame({
    'Model': names,
    'Score': scores,
    'Std Dev': stdevs,
    'Time (ms)': times,
    'Estimator': estimators})

results_df.sort_values(by='Score', ascending=False)

,Model,Score,Std Dev,Time (ms),Estimator
5,DT,97.945102,0.210043,1192,"DecisionTreeClassifier(class_weight=None, crit..."
6,ABDT,97.646607,0.259626,2733,"AdaBoostClassifier(algorithm='SAMME.R',\n ..."
9,XGB,97.485925,0.341441,13030,"XGBClassifier(base_score=0.5, booster='gbtree'..."
0,LR,95.258851,0.207678,12488,"LogisticRegression(C=1.0, class_weight=None, d..."
1,LDA,95.247350,0.298633,1222,"LinearDiscriminantAnalysis(n_components=None, ..."
3,GNB,95.235895,0.370420,778,"GaussianNB(priors=None, var_smoothing=1.0)"
7,SGD,92.228188,1.308504,5531,"SGDClassifier(alpha=0.0001, average=False, cla..."
8,PCT,85.032556,9.424936,1158,"Perceptron(alpha=0.0001, class_weight=None, ea..."
4,BNB,50.522470,0.759787,698,"BernoulliNB(alpha=1.0, binarize=0.0, class_pri..."
2,QDA,50.235404,0.645967,939,"QuadraticDiscriminantAnalysis(priors=None, reg..."


In [110]:
model = models['DT'][0]
model

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [111]:
# treinar o melhor modelo com todos os dados
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [112]:
# definir arquivo Pickle de entrada
#arquivo = 'afastamentos-nomes-sexo-preparado.pkl.bz2'
arquivo = 'cadastro-nomes-sexo-preparado.pkl.bz2'
#arquivo = 'extra/nome_sexo_pf05.csv'

df2 = pd.read_pickle(arquivo)
df2.head()

,SEXO,L0,L1,L2,L3,L4,L5,L6,L7,L8,L9,L10,L11,L12,L13,L14,L15
PNOME,,,,,,,,,,,,,,,,,
MARIA,0,1,9,18,1,13,0,0,0,0,0,0,0,0,0,0,0
JOSE,1,5,19,15,10,0,0,0,0,0,0,0,0,0,0,0,0
ANA,0,1,14,1,0,0,0,0,0,0,0,0,0,0,0,0,0
JOAO,1,15,1,15,10,0,0,0,0,0,0,0,0,0,0,0,0
CARLOS,1,19,15,12,18,1,3,0,0,0,0,0,0,0,0,0,0


In [113]:
df2.shape

(48970, 17)

In [114]:
X_test = df2.drop(['SEXO'], axis=1) # tudo, exceto a coluna alvo
y_test = df2['SEXO'] # apenas a coluna alvo

In [115]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 0, ..., 1, 1, 1], dtype=uint8)

In [116]:
from sklearn.metrics import accuracy_score

print("accuracy score: %.2f %%" % (accuracy_score(y_test, y_pred) * 100))

accuracy score: 83.37 %


In [117]:
from sklearn.metrics import confusion_matrix

print("confusion matrix:\n", confusion_matrix(y_test, y_pred))

confusion matrix:
 [[21702   189     0]
 [  133 19126     0]
 [ 4043  3777     0]]


In [118]:
dados = pd.DataFrame({'REAL': y_test, 'PREV': y_pred}, index=X_test.index)
for col in dados.columns:
    dados[col] = dados[col].map({0: 'F', 1: 'M', 9: 'X'})
dados.head()

,REAL,PREV
PNOME,,
MARIA,F,F
JOSE,M,M
ANA,F,F
JOAO,M,M
CARLOS,M,M


In [119]:
dados[dados['REAL'] != dados['PREV']].head(20)

,REAL,PREV
PNOME,,
JORGE,M,F
RAQUEL,F,M
LILIAN,F,M
ISABEL,F,M
MIRIAM,F,M
VIVIAN,F,M
GEORGE,M,F
MONIQUE,F,M
MIRIAN,F,M
